### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
7454,2023-11-19,Brasil Nbb,11:00,R10 Score Vasco da Gama,Brasília,1.26,3.75,148.5,1.88,1.88,-9.5,1.95,1.76
7455,2023-11-19,Espanha Acb,08:30,Granada,MoraBanc Andorra,1.92,1.92,167.5,1.81,1.95,-1.5,2.06,1.75
7456,2023-11-19,Espanha Acb,08:30,Manresa,Obradoiro CAB,1.48,2.75,169.5,1.82,1.99,-5.5,2.02,1.74
7457,2023-11-19,Espanha Acb,08:30,Real Madrid,Unicaja,1.22,4.56,169.5,1.83,1.95,-9.5,2.01,1.77
7458,2023-11-19,Espanha Acb,13:00,Baskonia,Valencia,1.58,2.46,162.5,1.81,1.95,-5.5,2.07,1.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7641,2023-11-19,Turquia Kbsl Feminina,08:00,Hatay F,Tarsus F,5.61,1.12,163.5,1.88,1.79,11.5,1.97,1.74
7642,2023-11-19,Turquia Kbsl Feminina,09:00,Galatasaray F,Bellona Kayseri F,1.09,6.62,151.5,1.84,1.84,-14.5,1.95,1.76
7643,2023-11-19,Turquia Kbsl Feminina,10:00,Izmit Bld. F,Besiktas F,2.51,1.49,146.5,1.84,1.84,3.5,1.99,1.73
7644,2023-11-19,Turquia Kbsl Feminina,11:00,Bursa F,Emlak Konut F,1.78,1.95,157.5,1.83,1.83,-2.5,2.04,1.69


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
69,14:00,Eua Ncaa,Southern Miss,Utah Valley,137.5,1.91,0.8681,Over
79,20:00,Eua Ncaa,Arizona,Texas Arlington,154.5,1.91,0.9186,Over
